# Understanding the Amazon rainforest from space

##Overview of Section
1. Split of test data/training data Justification
2. Weather models (Multi-classifier single labels)
3. Single Label Land Approach
4. Multi-label land approach
5. Combined Land/Weather approach


Given we are mostly interested in being able to tell where deforestation is occuring, we will simplify the land tags into naturally occuring phenomena 



In [5]:
import numpy as np # linear algebra
import pandas as pd 
from subprocess import check_output
import matplotlib.pyplot as plt
%matplotlib inline
import scipy
import scipy.stats
import seaborn as sns
import cv2
import plotly.offline as py
py.init_notebook_mode(connected=True)

import plotly.graph_objs as go
import plotly.tools as tls
import random
import math
# from osgeo import gdal
from skimage import io
#import imageio
from sklearn import preprocessing

from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import LogisticRegression
from skimage import transform
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.cross_validation import StratifiedKFold, cross_val_score, train_test_split 
from sklearn.preprocessing import MultiLabelBinarizer, MinMaxScaler
from sklearn.metrics import fbeta_score
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.multioutput import ClassifierChain
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings('ignore')

## 1. Split of test training data
We are choosing to split the data 90 to 10 training to test. This is because 

We are including so much more data in the training set because some of the features are 

Nick/Wei please build in random shuffle of the data here.

In [18]:
df = pd.read_csv('train_labels.csv')
def apply_land(land_tags):
    land_list=[]
    other_tags=['habitation','bare_ground','cultivation','agriculture','blow_down', 'conventional_mine', 'selective_logging', 'slash_burn', 'artisinal_mine', 'blooming']
    if 'primary' in land_tags:
        land_list.append('primary')
    if 'water' in land_tags:
        land_list.append("water")
    other_present=0
    for i in other_tags:
        if i in land_tags:
            other_present=1
            break
    if other_present==1:
        land_list.append('other')
    if land_list == []:
        land_list.append('none')
    return ' '.join(land_list)

#Add a column for weather tags
def apply_weather(weather):
    if 'partly_cloudy' in weather:
        return 'partly_cloudy'
    elif 'cloudy' in weather:
        return 'cloudy'
    elif 'haze' in weather:
        return 'haze'
    else:
        return 'clear'
    
def add_two_columns(df):
    df['weather'] = df.tags.map(apply_weather)
    df['land'] = df.tags.map(apply_land)
    return df

add_two_columns(df)
df.to_csv("train_labels_clean.csv")

In [25]:
#obtain processed training labels and data
labels = pd.read_csv('train_labels_clean.csv')
data = pd.read_csv('train_data.csv')

train_labels_weather=labels.weather[:39000]
train_labels_land=labels.land[:39000]
train_data=data[:39000]

test_labels_weather=df.weather[39000:]
test_labels_land=df.land[39000:]
test_data=data[39000:]

## Weather feature identification

Weather feature identification is being performed for cloud and haze removal. This facilitates the successive water, land and primary feature identification.

1. Create Useful fucntions
2. Try a variety of models
3. Improve on the most successful model preformance

In [26]:
def cross_validation(model, data, labels):
    scores = cross_val_score(model, data, labels)
    print("Cross-Validation\n Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2)) 

In [27]:
def confusion_mat(y_true, y_pred):
    print('Confusion Matrix')
    print(confusion_matrix(y_true, y_pred))

 ## 1. Logistic Regresion model
 

In [30]:
#Logistic Regression
def logistic_regression(training_data, training_labels, testing_data, testing_labels):
    model =LogisticRegression()
    model.fit(training_data, training_labels)
    model_predictions=model.predict(testing_data)
    print('Logistic Regression: ' + classification_report(testing_labels,model_predictions))
    cross_validation(model, training_data, training_labels)
    confusion_mat(testing_labels, model_predictions)
    #classifier_chain(model, train_data_weather, train_label_weather, test_data_weather, test_label_weather)
logistic_regression(train_data, train_labels_weather, test_data, test_labels_weather)

Logistic Regression:                precision    recall  f1-score   support

        clear       0.86      0.96      0.91      1047
       cloudy       0.63      0.58      0.61        69
         haze       0.68      0.44      0.54        97
partly_cloudy       0.83      0.61      0.70       266

  avg / total       0.84      0.84      0.83      1479

Cross-Validation
 Accuracy: 0.82 (+/- 0.01)
Confusion Matrix
[[1000    6   12   29]
 [  19   40    7    3]
 [  40   13   43    1]
 [  99    4    1  162]]


THis model works okay for clear weather - but not really for anything else. Log Reg probably not the way to go.

Which labels is it confusing? Everything for Clear by the look of if

In [33]:
def dec_tree_model(maxs,data_train,labels_train,data_test,labels_test):
    #set the model to take in the k value
    model = DecisionTreeClassifier(min_samples_leaf=20, max_depth=maxs)
    #fit per the train data
    model.fit(data_train, labels_train)
    model_predictions=model.predict(data_test)
    #set by the test data
#     score = model.score(data_test, labels_test)
    #print the score prettily
    return classification_report(labels_test,model_predictions), model, model_predictions
for i in [5,10,20,50,100]:
    print("Decision Tree: K-branch levels " + str(i))
    classification_rpt, model, model_predictions = dec_tree_model(i,train_data,train_labels_weather,test_data,test_labels_weather)
    print(classification_rpt)
    cross_validation(model, train_data, train_labels_weather)
    confusion_mat(test_labels_weather, model_predictions)

Decision Tree: K-branch levels 5
               precision    recall  f1-score   support

        clear       0.87      0.96      0.92      1047
       cloudy       0.66      0.68      0.67        69
         haze       0.79      0.45      0.58        97
partly_cloudy       0.83      0.62      0.71       266

  avg / total       0.85      0.85      0.84      1479

Cross-Validation
 Accuracy: 0.84 (+/- 0.00)
Confusion Matrix
[[1008    6    5   28]
 [  11   47    6    5]
 [  40   12   44    1]
 [  95    6    1  164]]
Decision Tree: K-branch levels 10
               precision    recall  f1-score   support

        clear       0.92      0.95      0.93      1047
       cloudy       0.71      0.70      0.70        69
         haze       0.77      0.53      0.63        97
partly_cloudy       0.80      0.77      0.79       266

  avg / total       0.88      0.88      0.88      1479

Cross-Validation
 Accuracy: 0.87 (+/- 0.01)
Confusion Matrix
[[997   4   8  38]
 [  6  48   5  10]
 [ 30  14  51 

In [34]:
def random_forest_model(maxs,data_train,labels_train,data_test,labels_test):
    #set the model to take in the k value
    model = RandomForestClassifier(n_estimators=20, max_depth=maxs, min_samples_leaf=2)
     #fit per the train data
    model.fit(data_train, labels_train)
    model_predictions=model.predict(data_test)
    #set by the test data
#     score = model.score(data_test, labels_test)
    #print the score prettily
    return classification_report(labels_test,model_predictions), model, model_predictions

for i in [5,10,20,50,75]:
    print("Random Forest: K-branch levels " + str(i))
    classification_rpt, model,model_predictions = random_forest_model(i,train_data,train_labels_weather,test_data,test_labels_weather)
    print(classification_rpt)
    cross_validation(model, train_data, train_labels_weather)
    confusion_mat(test_labels_weather, model_predictions)
#     print('One Vs. Rest K-branch levels ' + str(i))
#     classifier_chain(model, train_data, train_labels, test_data, test_labels)

Random Forest: K-branch levels 5
               precision    recall  f1-score   support

        clear       0.86      0.98      0.92      1047
       cloudy       0.72      0.62      0.67        69
         haze       0.84      0.47      0.61        97
partly_cloudy       0.86      0.59      0.70       266

  avg / total       0.86      0.86      0.85      1479

Cross-Validation
 Accuracy: 0.84 (+/- 0.01)
Confusion Matrix
[[1022    5    5   15]
 [  12   43    4   10]
 [  40   11   46    0]
 [ 108    1    0  157]]
Random Forest: K-branch levels 10
               precision    recall  f1-score   support

        clear       0.90      0.98      0.94      1047
       cloudy       0.84      0.67      0.74        69
         haze       0.85      0.58      0.69        97
partly_cloudy       0.90      0.73      0.80       266

  avg / total       0.89      0.89      0.89      1479

Cross-Validation
 Accuracy: 0.89 (+/- 0.00)
Confusion Matrix
[[1027    2    6   12]
 [  10   46    4    9]
 [  33

In [38]:
sel_weather_model = RandomForestClassifier(n_estimators=50, max_depth=20)
     #fit per the train data
sel_weather_model.fit(train_data, train_labels_weather)
model_predictions=sel_weather_model.predict(test_data)
print(classification_report(test_labels_weather,model_predictions))
cross_validation(model, train_data, train_labels_weather)
confusion_mat(test_labels_weather, model_predictions)

               precision    recall  f1-score   support

        clear       0.92      0.98      0.95      1047
       cloudy       0.86      0.70      0.77        69
         haze       0.82      0.63      0.71        97
partly_cloudy       0.92      0.79      0.85       266

  avg / total       0.91      0.91      0.91      1479

Cross-Validation
 Accuracy: 0.90 (+/- 0.00)
Confusion Matrix
[[1026    2    8   11]
 [  11   48    4    6]
 [  30    5   61    1]
 [  53    1    1  211]]


This is now a fairly accurate model for the clear and partly cloudy models. Both cloudy and hazy are getting confused with clear.

## 2. Land Models - single label models

In [ ]:
def random_forest_model(maxs,data_train,labels_train,data_test,labels_test):
    #set the model to take in the k value
    model = RandomForestClassifier(n_estimators=20, max_depth=maxs, min_samples_leaf=2)
     #fit per the train data
    model.fit(data_train, labels_train)
    model_predictions=model.predict(data_test)
    #set by the test data
#     score = model.score(data_test, labels_test)
    #print the score prettily
    return classification_report(labels_test,model_predictions), model, model_predictions

for i in [5,10,20,50,75,100]:
    print("Random Forest: K-branch levels " + str(i))
    classification_rpt, model, model_predictions = random_forest_model(i,train_data,train_labels_land,test_data,test_labels_land)
    print(classification_rpt)
    cross_validation(model, train_data, train_labels_weather)
    confusion_mat(test_labels_weather, model_predictions)
#     print('One Vs. Rest K-branch levels ' + str(i))
#     classifier_chain(model, train_data, train_labels, test_data, test_labels)

Random Forest: K-branch levels 5
                     precision    recall  f1-score   support

               none       0.70      0.65      0.68        69
              other       0.00      0.00      0.00        21
            primary       0.83      0.85      0.84       722
      primary other       0.58      0.81      0.68       415
      primary water       0.61      0.24      0.34       126
primary water other       0.54      0.25      0.35       114
              water       0.00      0.00      0.00         8
        water other       0.00      0.00      0.00         4

        avg / total       0.70      0.71      0.69      1479

Cross-Validation
 Accuracy: 0.85 (+/- 0.01)
Confusion Matrix
[[  0   0   0   6   0 592 358  45  44   2]
 [  0   0   0  45   0   6  17   1   0   0]
 [  0   0   0  12   0  61  21   1   2   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   1   0  73 182   2   8   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0  

The model does extremely badly on picking up models with water - it is fairly efficient at picking up primar and primary tags vs other

In [ ]:
sel_land_model = RandomForestClassifier(n_estimators=50, max_depth=100, min_samples_leaf=5)
     #fit per the train data
sel_land_model.fit(train_data_land, train_label_land)
model_predictions=sel_land_model.predict(test_data_land)
print(classification_report(test_label_land,model_predictions))
cross_validation(model, train_data_weather, train_label_weather)
confusion_mat(test_label_weather, model_predictions)

This is clearly not a good model, as by forcing the labels to separate we lose all the predictive power contained in other labels. I.e. what's in a primary water image, will hopefully help use to predict whats in a water other tag as well.

# Land Models - multi-label models

In [ ]:
train_data, test_data, train_label, test_label = train_test_split(combined_matrix, df['tags'].tolist(), test_size=0.1)
# Create training and test sets for weather labels by randomly splitting the original dataset as 90%/10%
train_data_weather, test_data_weather, train_label_weather, test_label_weather = train_test_split(combined_matrix, df['weather'].tolist(), test_size=0.1)
# Create training and test sets for land labels by randomly splitting the original dataset as 90%/10%
train_data_land, test_data_land, train_label_land, test_label_land = train_test_split(combined_matrix, df['land'].tolist(), test_size=0.1)

In [203]:
df['tags_split'] = df['land'].map(lambda row: row.split(" "))
mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(df['tags_split'])
train_data_land_multi, test_data_land_multi, train_label_land_multi, test_label_land_multi= train_test_split(combined_matrix, labels, test_size=0.1)

In [204]:
def ovr_model(estimator, data_train,labels_train,data_test,labels_test):
    model=OneVsRestClassifier(estimator).fit(train_data_land_multi,train_label_land_multi)
    model_predict=model.predict(test_data_land_multi)
    print('OVR with ' + str(model.get_params().get('estimator'))[0:str(model.get_params().get('estimator')).find('(')]+' estimator: ')
    print(classification_report(test_label_land_multi,model_predict))
    cross_validation(model, train_data_land_multi, train_label_land_multi)
    confusion_mat(test_label_land_multi.argmax(axis=1), model_predict.argmax(axis=1))
ovr_model(RandomForestClassifier(n_estimators=50,max_depth=50, min_samples_leaf=2), train_data_land_multi,train_label_land_multi,test_data_land_multi,test_label_land_multi)
ovr_model(DecisionTreeClassifier(max_depth=50, min_samples_leaf=10), train_data_land_multi,train_label_land_multi,test_data_land_multi,test_label_land_multi)

NameError: name 'train_data' is not defined

In [ ]:
def classifier_chain_model(estimator, data_train,labels_train,data_test,labels_test):
    model = ClassifierChain(estimator).fit(train_data_land_multi, train_label_land_multi)
    model_predict=cc.predict(test_data_land_multi)
    print('Classifier Chain with ' + str(model.get_params().get('base_estimator'))[0:str(model.get_params().get('base_estimator')).find('(')]+' base estimator: ')
    print(classification_report(test_label_land_multi,model_predict))
    cross_validation(model, train_data_land_multi, train_label_land_multi)
    confusion_mat(test_label_land_multi.argmax(axis=1), model_predict.argmax(axis=1))
classifier_chain_model(RandomForestClassifier(n_estimators=50,max_depth=50, min_samples_leaf=2), train_data_land_multi,train_label_land_multi,test_data_land_multi,test_label_land_multi)
classifier_chain_model(DecisionTreeClassifier(max_depth=50, min_samples_leaf=10), train_data_land_multi,train_label_land_multi,test_data_land_multi,test_label_land_multi)

In [ ]:
def k_neighbors_model(k,data_train,labels_train,data_test,labels_test):
    model = KNeighborsClassifier(n_neighbors=k, algorithm='ball_tree').fit(train_data_land_multi, train_label_land_multi)
    model_predict=nbrs.predict(test_data_land_multi)
    print(classification_report(test_label_land_multi,nbrs_predict))
    cross_validation(model, train_data_land_multi, train_label_land_multi)
    confusion_mat(test_label_land_multi.argmax(axis=1), model_predict.argmax(axis=1))
for i in [1,5,10,12,13]:
    print("K-Neighbors: " + str(i))
    k_neighbors_model(i,train_data_land_multi,train_label_land_multi,test_data_land_multi,test_label_land_multi)   

The model is now doing a better job at picking up the rare labels, though it struggles a lot with the water label. 
It is mostly good at detecting the primary label because it appears so often, though it does a relatively good job with the other label, and the none label (which should mostly be used for cloudly image.

The model does well at picking clear, cloudy, primary and not much else. To better identify roads, and water edge extraction may 
help. We're also not taking advantage of correlation with the model here.

Interestingly the model isn't picking too many false positives (i.e. precision), but is picking a lot of false negatives(i.e. recall). This suggests we need to build more features to pick up land masses, roads and waters. 


Out of scope
Try Convultional Neural Networks
Ask June for help in Set-up